In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import torch
import timewarp_lib.vector_timewarpers as vtw
import torch.nn as nn
import sys
sys.path.append("..")
import comparison_models.model_applier as ma

In [ ]:
test_time_dtw_vector_timewarper = vtw.DTWVectorTimewarper()

In [ ]:
# we trained on augmented data, but we should just apply to regular data

DATAFILE=f"../data/trainTest2DLetterAScaled.npz"
MODELFOLDERGLOB= "../results/augpca/*/savedmodel"
data = np.load(DATAFILE)
test = data["test"]
train = data["train"]

num_trains, num_ts, channels = train.shape
num_tests, num_ts, channels = test.shape

In [ ]:
for modelname in glob.glob(MODELFOLDERGLOB):
    model = ma.ModelApplier(modelname)
    print(modelname)
    print(model.latent_dim)

In [ ]:
results = []
for modelname in glob.glob(MODELFOLDERGLOB):
    model = ma.ModelApplier(modelname)
    if model.latent_dim > 16:
        continue
    print(model.latent_dim)
    recon_test = model.apply(model.embed(test))
    recon_train = model.apply(model.embed(train))
    train_dtw_recon, train_dtw_actual = test_time_dtw_vector_timewarper.timewarp_first_and_second(
        torch.tensor(recon_train,dtype=torch.float), 
        torch.tensor(train,dtype=torch.float))
    train_aligned_loss = np.sqrt(
        nn.functional.mse_loss(train_dtw_recon, train_dtw_actual, reduction="sum").detach().numpy()
        / (num_ts * num_trains))
    test_dtw_recon, test_dtw_actual = test_time_dtw_vector_timewarper.timewarp_first_and_second(
        torch.tensor(recon_test,dtype=torch.float), 
        torch.tensor(test,dtype=torch.float))
    test_aligned_loss = np.sqrt(
        nn.functional.mse_loss(test_dtw_recon, test_dtw_actual, reduction="sum").detach().numpy()
        / (num_ts * num_tests))
    test_error = np.sqrt(np.sum(np.square(recon_test - test))/(num_ts * num_tests))
    train_error = np.sqrt(np.sum(np.square(recon_train - train))/(num_ts * num_trains))
    results.append((model.latent_dim,train_aligned_loss,test_aligned_loss, train_error, test_error))
results = np.array(results)
np.save("pca_results.npy",results)

In [ ]:
results

In [ ]:
results = np.load("pca_results.npy")
plt.figure(figsize=(6,6))
plt.scatter(results[:,1],results[:,2])
plt.scatter(results[:,3],results[:,4])
plt.axis("equal")
plt.xlim(0.0,0.4)
plt.ylim(0.0,0.4)

In [ ]:
for modelname in glob.glob(MODELFOLDERGLOB):
    model = ma.ModelApplier(modelname)
    if model.latent_dim != 16:
        continue
    for dim in range(16):
        plt.figure(figsize=(8,8))
        for latent in np.linspace(-0.01,0.01,5):
            latents = np.zeros((1,16))
            latents[0,dim] = latent
            traj = model.apply(latents)
            plt.plot(traj[0,:,0],traj[0,:,1])
            plt.axis("equal")
            plt.xlim(-4,4)
            plt.ylim(-4,4)

In [ ]:
plt.figure(figsize=(15,15))
for modelname in glob.glob(MODELFOLDERGLOB):
    model = ma.ModelApplier(modelname)
    if model.latent_dim!=16:
        continue
    recon_train = model.apply(model.embed(train))
    for i in range(5):
        plt.scatter(train[i,:,0]+i*5,train[i,:,1])
        plt.plot(recon_train[i,:,0]+i*5,recon_train[i,:,1],c="k")
plt.axis("equal")